In [ ]:
{
    "name": "ShoppingMart_DI_BL",
    "objectId": "3a3bb860-60ea-4fa7-ad50-bc1840001df6",
    "properties": {
        "activities": [
            {
                "name": "Filter1",
                "type": "Filter",
                "dependsOn": [
                    {
                        "activity": "Git_To_Fabric",
                        "dependencyConditions": [
                            "Succeeded"
                        ]
                    }
                ],
                "typeProperties": {
                    "items": {
                        "value": "@json(activity('Git_To_Fabric').output.Response)",
                        "type": "Expression"
                    },
                    "condition": {
                        "value": "@or(\n    contains(toLower(item().sink_file), 'orders.csv'),\n    or(\n        contains(toLower(item().sink_file), 'customers.csv'),\n        contains(toLower(item().sink_file), 'products.csv')\n    )\n)",
                        "type": "Expression"
                    }
                }
            },
            {
                "name": "Git_To_Fabric",
                "type": "WebActivity",
                "dependsOn": [],
                "policy": {
                    "timeout": "0.12:00:00",
                    "retry": 0,
                    "retryIntervalInSeconds": 30,
                    "secureOutput": false,
                    "secureInput": false
                },
                "typeProperties": {
                    "method": "GET",
                    "relativeUrl": "refs/heads/main/ShoppingMart_StructuredMetaData.json"
                },
                "externalReferences": {
                    "connection": "47d72662-40eb-46bb-acca-481659aa558d"
                }
            },
            {
                "name": "FileArray",
                "type": "SetVariable",
                "dependsOn": [
                    {
                        "activity": "Filter1",
                        "dependencyConditions": [
                            "Succeeded"
                        ]
                    }
                ],
                "policy": {
                    "secureOutput": false,
                    "secureInput": false
                },
                "typeProperties": {
                    "variableName": "FileArray",
                    "value": {
                        "value": "@activity('Filter1').output.value",
                        "type": "Expression"
                    }
                }
            },
            {
                "name": "ForEach1",
                "type": "ForEach",
                "dependsOn": [
                    {
                        "activity": "FileArray",
                        "dependencyConditions": [
                            "Succeeded"
                        ]
                    }
                ],
                "typeProperties": {
                    "items": {
                        "value": "@activity('Filter1').output.value",
                        "type": "Expression"
                    },
                    "isSequential": false,
                    "activities": [
                        {
                            "name": "Copy data1",
                            "type": "Copy",
                            "dependsOn": [],
                            "policy": {
                                "timeout": "0.12:00:00",
                                "retry": 0,
                                "retryIntervalInSeconds": 30,
                                "secureOutput": false,
                                "secureInput": false
                            },
                            "typeProperties": {
                                "source": {
                                    "type": "DelimitedTextSource",
                                    "storeSettings": {
                                        "type": "HttpReadSettings",
                                        "requestMethod": "GET"
                                    },
                                    "formatSettings": {
                                        "type": "DelimitedTextReadSettings"
                                    },
                                    "datasetSettings": {
                                        "annotations": [],
                                        "type": "DelimitedText",
                                        "typeProperties": {
                                            "location": {
                                                "type": "HttpServerLocation",
                                                "relativeUrl": {
                                                    "value": "@item().source_url",
                                                    "type": "Expression"
                                                }
                                            },
                                            "columnDelimiter": ",",
                                            "escapeChar": "\\",
                                            "firstRowAsHeader": true,
                                            "quoteChar": "\""
                                        },
                                        "schema": [],
                                        "externalReferences": {
                                            "connection": "39bc2584-ea86-499a-955a-6748f20520ee"
                                        }
                                    }
                                },
                                "sink": {
                                    "type": "DelimitedTextSink",
                                    "storeSettings": {
                                        "type": "LakehouseWriteSettings"
                                    },
                                    "formatSettings": {
                                        "type": "DelimitedTextWriteSettings",
                                        "fileExtension": ".txt"
                                    },
                                    "datasetSettings": {
                                        "annotations": [],
                                        "connectionSettings": {
                                            "name": "shoppingmart_Bronze",
                                            "properties": {
                                                "annotations": [],
                                                "type": "Lakehouse",
                                                "typeProperties": {
                                                    "workspaceId": "87269aa4-ec56-4361-8f62-77718f31b075",
                                                    "artifactId": "9e29ae77-4a12-4321-8ec1-9990a923ac42",
                                                    "rootFolder": "Files"
                                                },
                                                "externalReferences": {
                                                    "connection": "9a10fe8c-48b8-4700-88a0-d76c18de20cf"
                                                }
                                            }
                                        },
                                        "type": "DelimitedText",
                                        "typeProperties": {
                                            "location": {
                                                "type": "LakehouseLocation",
                                                "fileName": {
                                                    "value": "@item().sink_file",
                                                    "type": "Expression"
                                                },
                                                "folderPath": {
                                                    "value": "@item().sink_folder",
                                                    "type": "Expression"
                                                }
                                            },
                                            "columnDelimiter": ",",
                                            "escapeChar": "\\",
                                            "firstRowAsHeader": true,
                                            "quoteChar": "\""
                                        },
                                        "schema": []
                                    }
                                },
                                "enableStaging": false,
                                "translator": {
                                    "type": "TabularTranslator",
                                    "typeConversion": true,
                                    "typeConversionSettings": {
                                        "allowDataTruncation": true,
                                        "treatBooleanAsNumber": false
                                    }
                                }
                            }
                        }
                    ]
                }
            }
        ],
        "variables": {
            "FileArray": {
                "type": "Array"
            }
        },
        "lastModifiedByObjectId": "2fb1483d-4fb2-4642-9890-a8dab99a2fc4",
        "lastPublishTime": "2025-10-29T16:07:17Z"
    }
}